<a href="https://colab.research.google.com/github/LovedeepB/211/blob/master/colab/Analysis_approach2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# REMOVING CONFOUNDERS


Will try two different strategies to try and remove the confounders using the GAN Model that is described in the paper that is linked below.   
   - https://www.nature.com/articles/s41467-020-19784-9#Sec2

In [1]:
# getting the sample data from kaggle 
!pip install -q kaggle 

In [ ]:
from google.colab import files

# upload the kaggle.json file
files.upload()

In [3]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d lovedeepbajaj/sample-data

 91% 105M/115M [00:03<00:00, 50.6MB/s] 
100% 115M/115M [00:03<00:00, 35.3MB/s]


In [5]:
# unziping the dataset 
!unzip sample-data.zip

Archive:  sample-data.zip
  inflating: data/test_data/dataset_1/negative/00000001_000.png  
  inflating: data/test_data/dataset_1/negative/00000001_001.png  
  inflating: data/test_data/dataset_1/negative/00000001_002.png  
  inflating: data/test_data/dataset_1/negative/00000002_000.png  
  inflating: data/test_data/dataset_1/negative/00000003_000.png  
  inflating: data/test_data/dataset_1/negative/00000003_001.png  
  inflating: data/test_data/dataset_1/negative/00000003_002.png  
  inflating: data/test_data/dataset_1/negative/00000003_003.png  
  inflating: data/test_data/dataset_1/negative/00000003_004.png  
  inflating: data/test_data/dataset_1/negative/00000003_005.png  
  inflating: data/test_data/dataset_1/positive/00000003_006.png  
  inflating: data/test_data/dataset_1/positive/00000003_007.png  
  inflating: data/test_data/dataset_1/positive/00000004_000.png  
  inflating: data/test_data/dataset_1/positive/00000005_000.png  
  inflating: data/test_data/dataset_1/positive/000

In [6]:
# importing in required libraries 
import cv2 
import seaborn as sns 
from os import listdir
from os.path import join
import matplotlib.pyplot as plt 

In [34]:
# this function loads in the images and returns them to you as well as the label (covid-19 pos or covid-19 neg)
# ONLY LOADING IN 10 IMAGES 
def get_images(image_dir, category, size, x_rays):
    for img in listdir(image_dir)[:10]:
        # getting the location of that image 
        location = join(image_dir, img)
        
        # reading in the image as a gray scale 
        temp = cv2.imread(location, cv2.IMREAD_GRAYSCALE)
        
        # resizing the image
        temp = cv2.resize(temp, size)
        
        # normalizing the image 
        temp = temp/255
        
        # storing the image and the label of that image inside of data 
        x_rays.append([temp, category])

In [35]:
# location of the images for dataset 1
dataset1_neg_location = "data/test_data/dataset_1/negative"
dataset1_pos_location = "data/test_data/dataset_1/positive"

In [36]:
# will contain all of the x-rays
data = []

# the size of the image 
IMG_SIZE = (200, 200)


# READING IN NEGATIVE IMAGES FOR DATASET 1
get_images(dataset1_neg_location, 0, IMG_SIZE, data)
    

# READING IN POSITIVE IMAGES FOR DATASET 1
get_images(dataset1_pos_location, 1, IMG_SIZE, data)


# seeing how many pictures we have for dataset 1
len(data)

20

In [37]:
# seeing how many non covid-19 related images do we have 
len([img for img in data if img[1] == 0])

10

In [38]:
# seeing how many covid-19 related images do we have 
len([img for img in data if img[1] == 1])

10

In [39]:
# location of the images for dataset 2
dataset2_neg_location = "data/test_data/dataset_2/negative"
dataset2_pos_location = "data/test_data/dataset_2/positive"

In [40]:
data2 = []
# READING IN THE NEGATIVE IMAGES FOR DATASET 2
get_images(dataset2_neg_location, 0, IMG_SIZE, data2)


# READING IN THE POSITIVE IMAGES FOR DATASET 2
get_images(dataset2_pos_location, 1, IMG_SIZE, data2)

len(data2)

20

In [41]:
# seeing how many non covid-19 related images do we have 
len([img for img in data2 if img[1] == 0])

10

In [42]:
# seeing how many covid-19 related images do we have 
len([img for img in data2 if img[1] == 1])

10

<br /> 

## METHOD 1

__REMOVING BIAS ON THE DATA SOURCE LEVEL__   

First thing to do is to figure out how the GAN Model that we are using works. This will allow us to determine what needs to be edited and what doesn't. 

In [43]:
!pip install dcor

In [44]:
# importing the required libraries 

# will need for creating the confounder prediction model or the confounder-free prediction model 
from keras.models import Sequential, Model
from keras.layers import Activation, Dense, Flatten, Input, Layer, MaxPooling2D, Conv2D, Reshape
from keras.losses import binary_crossentropy
from keras import backend as K
from tensorflow.keras.optimizers import Adam # - Works
import tensorflow as tf

# will need to create an array of zeros when training the GAN 
import numpy as np

# using to read in the required files
import pandas as pd
import glob 

# using for the distance correlation calculation
import dcor


import sys
import random 

In [45]:
# adversial loss for squared correlation (the one that the code uses by default)
def correlation_coefficient_loss(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(tf.multiply(xm,ym))
    r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym)))) + 1e-5
    r = r_num / r_den

    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [46]:
# adversial loss for mse
def inv_mse(y_true, y_pred):
    mse_value = K.sum(K.square(y_true-y_pred))

    return -mse_value

In [79]:
class GAN():
    
    def __init__(self):
        
        # three different types of Adam optimizers
        optimizer = Adam(0.0001) # for the actual classification (after removing confounders)
        optimizer_distiller = Adam(0.0002) # when we remove the confounders
        optimizer_regressor = Adam(0.0002) # for the model that predicts the confounders

        # THIS PORTION IS FOR PREDICTING THE CONFOUNDERS 
        # Build a naive feature encoder
        input_image = Input(shape=(200,200,1), name='input_image')
        
        feature_dense_enc = Conv2D(2, activation='tanh', kernel_size=(3,3),padding='valid')(input_image)
        feature_dense_enc = MaxPooling2D(pool_size=(2,2))(feature_dense_enc)
        feature_dense_enc = Conv2D(4, activation='tanh', kernel_size=(3,3),padding='valid')(feature_dense_enc)
        feature_dense_enc = MaxPooling2D(pool_size=(2,2))(feature_dense_enc)
        feature_dense_enc = Conv2D(8, activation='tanh', kernel_size=(3,3),padding='valid')(feature_dense_enc)
        feature_dense_enc = MaxPooling2D(pool_size=(2,2))(feature_dense_enc)
        
        feature_dense_enc = Flatten()(feature_dense_enc) 
        self.encoder = Model(input_image, feature_dense_enc)

        # Build and compile the bias predictor (encoder fixed)
        self.regressor = self.build_regressor()
        self.regressor.compile(loss='mse', optimizer=optimizer_regressor)
        
        
        # THIS PORTION IS FOR REMOVING THE CONFOUNDERS. ALSO USES THE BUILD_REGRESSOR FUNCTION TO REMOVE THE BIAS
        # For the distillation model (removing bias) we will only train the encoder adversarially
        self.regressor.trainable = False
        cf = self.regressor(feature_dense_enc)
        self.distiller = Model(input_image, cf)
        self.distiller.compile(loss=correlation_coefficient_loss, optimizer=optimizer_distiller)
        
        # use the following for MSE-based adversairal training
        #self.distiller.compile(loss=inv_mse, optimizer=optimizer_distiller)
        
        
        # THIS IS WHERE THE ACTUAL CONFOUNDER FREE CLASSIFICATION TAKES PLACE
        # Build and Compile the classifer (for the actual classification task) 
        input_feature_clf = Input(shape=(4232,), name='input_feature_dense') # needed to edit this 
        feature_clf = Dense(16, activation='tanh')(input_feature_clf)
        prediction_score = Dense(1, name='prediction_score')(feature_clf)
        self.classifier = Model(input_feature_clf, prediction_score)
        
        # Build the entire workflow
        prediction_score_workflow = self.classifier(feature_dense_enc)
        label_workflow = Activation('sigmoid', name='r_mean')(prediction_score_workflow)
        self.workflow = Model(input_image, label_workflow)
        self.workflow.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy'])

    # A FUNCTION USED BY THE BIAS PREDICTION MODEL 
    def build_regressor(self):
        inputs_x = Input(shape=(4232,)) # needed to edit this 
        feature = Dense(16, activation='tanh')(inputs_x)
        cf = Dense(1)(feature)

        return Model(inputs_x, cf)

    
    # THIS FUNCTION IS THE ONE WE CALL IN ORDER TO PASS IN THE REQUIRED DATA TO DO THE ANALYSIS 
    def train_cf(self, epochs, dataset1, dataset2, batch_size = 20, fold=0):
        # these are the images and the labels from dataset 1 and dataset 2 
        [train_img, label]  = dataset1
        [test_img, test_label] = dataset2
        
        
        # THIS WILL BE THE DISTANCE CORRELATION (FOR ME IT WILL BE ABOUT THE DIFFERENCE BETWEEN THE DATASETS)
        dc_cf = np.zeros((int(epochs/10)+1,))
        
        # THIS WILL RUN FOR THE # OF EPOCHS YOU SET 
        for epoch in range(epochs):

            # selecting half of the images from dataset 1 and half of the images from dataset 2 and then making the array random 
            combined_data = []


            # putting all of the images from dataset1 and dataset2 in the combined dataset 
            for img in train_img:
              combined_data.append(img)
            for img in test_img:
              combined_data.append(img)

            # since the number of images we use from the combined dataset and the dataset 1 have to be the same size we are going to suffle them so we don't just get all the images from dataset1 
            random.shuffle(combined_data)


            # reshaping the data 
            combined_data = np.array(combined_data).reshape(-1, 200, 200,1)
            data = np.array(train_img).reshape(-1, 200, 200,1)
            test_data = np.array(test_img).reshape(-1, 200, 200,1)



            
            # the images we will train the model on and teach it to remove these confounders
            training_feature_ctrl_batch = combined_data
            
            
            
            # ---------------------
            #  Train regressor (bias predictor)
            # ---------------------
            # flattening the dataset1 
            cf_ctrl_batch = self.encoder.predict(np.array(data))
            # flatten the combinedd_dataset images 
            encoded_feature_ctrl_batch = self.encoder.predict(training_feature_ctrl_batch[:20])
            r_loss = self.regressor.train_on_batch(encoded_feature_ctrl_batch, cf_ctrl_batch)



            # ---------------------
            #  Train Disstiller (bias removal)
            # ---------------------
            g_loss = self.distiller.train_on_batch(training_feature_ctrl_batch[:20], cf_ctrl_batch) 

                     
            # ---------------------
            #  Train Encoder & Classifier (actual classification task)
            # ---------------------
            # the actually classification will be done using these images -> since the combined dataset contains 20 images we can use the whole of dataset1 which contains 20 images 
            training_feature_batch = data
            # we want to predict covid vs non-covid for the actual classification
            dx_batch = np.array(label)
            # passing in dataset 1 images (training_feature_batch) as x and the label (covid vs non-covid) as dx_batch as y 
            c_loss = self.workflow.train_on_batch(training_feature_batch, dx_batch)
            
            # flipping the images 
            training_feature_batch = np.flip(training_feature_batch,1)
            training_feature_ctrl_batch = np.flip(training_feature_ctrl_batch,1)

            # repeat the process again but this time flip it 
            cf_ctrl_batch = self.encoder.predict(np.array(data))
            encoded_feature_ctrl_batch = self.encoder.predict(training_feature_ctrl_batch[:20])
            r_loss = self.regressor.train_on_batch(encoded_feature_ctrl_batch, cf_ctrl_batch)
            g_loss = self.distiller.train_on_batch(training_feature_ctrl_batch[:20], cf_ctrl_batch)
            c_loss = self.workflow.train_on_batch(training_feature_batch, dx_batch)
            
            
            # Plot the training progress
            if epoch % 10 == 0:
                c_loss_test_1 = self.workflow.evaluate(np.array(test_data), np.array(test_label), verbose = 0, batch_size = batch_size)
                
                # feature dist corr DO NOT UNDERSTAND THIS PORTION 
                #features_dense = self.encoder.predict(train_data[train_label == 0],  batch_size = batch_size)
                #dc_cf[int(epoch/10)] = dcor.u_distance_correlation_sqr(features_dense, train_data[train_label == 0])
                
                
                
                print ("%d [Acc: %f  Test Acc: %f]" % (epoch, c_loss[1], c_loss_test_1[1]))
                sys.stdout.flush()
                
        # returning the predictions   
        pred = self.workflow.predict(test_data)
        
        # return dist corr, learned features for both groups and the prediction and accuray progress of the model 
        return pred

In [80]:
gan_cf = GAN()

In [81]:
# getting the labels and the images separate 
train_data = []
train_label = []

for img in data:
    train_data.append(img[0])
    train_label.append(img[1])

test_data = []
test_label = []

for img2 in data2:
    test_data.append(img2[0])
    test_label.append(img2[1])

In [82]:
dataset1 = [train_data, train_label]
dataset2 = [test_data, test_label]


# getting the accuracy of the model 
y_pred = gan_cf.train_cf(epochs = 41, dataset1 = dataset1, dataset2= dataset2)

0 [Acc: 0.500000  Test Acc: 0.500000]
10 [Acc: 0.750000  Test Acc: 0.700000]
20 [Acc: 0.800000  Test Acc: 0.700000]
30 [Acc: 0.850000  Test Acc: 0.700000]
40 [Acc: 0.850000  Test Acc: 0.700000]


In [91]:
values = []

for prediction in y_pred:
  if prediction >= 0.50:
    values.append(1)
  else:
    values.append(0)

In [92]:
values

[1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]

In [93]:
test_label

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]